In [87]:
import matplotlib.pyplot as plt

import numpy as np

from sklearn.datasets import make_blobs
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import cross_validation

import pandas as pd



In [34]:
train_data_raw = pd.read_csv('train_enc.csv', quotechar='"', skipinitialspace=True)
cluster_set = pd.read_csv('variable_cluster.csv', quotechar='"', skipinitialspace=True)

In [128]:
train_data_raw[0:5]

,Unnamed: 0,TripType,VisitNumber,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS
0,0,999,5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,30,7,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2,26,8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,8,9,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,8,10,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
test_data_raw = pd.read_csv('test_enc.csv', quotechar='"', skipinitialspace=True)
test_data_raw[0:5]


,Unnamed: 0,VisitNumber,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,1-HR PHOTO,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS
0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,6,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
data = train_data_raw.drop(train_data_raw.columns[[0, 2, 6]], axis=1)
data[0:2]

,TripType,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS
0,999,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [147]:
data_1 = train_data_raw.drop(train_data_raw.columns[[0, 2, 6]], axis=1)

In [148]:
data_1[0:5]

,TripType,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS
0,999,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,26,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
data_test_cache = pd.read_csv('test_enc.csv', quotechar='"', skipinitialspace=True)
# drop row index, VisitNumber and Sunday
data_test_cache = data_test_cache.drop(data_test_cache.columns[[0, 1, 5]], axis=1)

In [157]:
data_test = pd.DataFrame(columns=list(data_1.columns))

In [158]:
data_test

,TripType,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS


In [159]:
for col_name in list(data_test.columns):
    try:
        data_test[col_name] = data_test_cache[col_name]
    except:
        data_test[col_name] = 0

In [160]:
data_test[0:2]

,TripType,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS
0,NaN,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,NaN,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
cluster_set[0:2]

,Unnamed: 0,var_name,var_index,cluster
0,1,X1.HR.PHOTO,7,cluster1
1,2,BAKERY,10,cluster1


In [5]:
def label_transform(col):
    le = pp.LabelEncoder()
    le.fit(col)
    label_seq = list(le.classes_)
    col_tr = le.fit_transform(col)
    return (label_seq, col_tr)



C:\Anaconda2\lib\site-packages\sklearn\preprocessing\label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Anaconda2\lib\site-packages\sklearn\preprocessing\label.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [64]:
cluster = list(set(cluster_set['cluster']))
cluster

['cluster10',
 'cluster8',
 'cluster9',
 'cluster2',
 'cluster3',
 'cluster1',
 'cluster6',
 'cluster7',
 'cluster4',
 'cluster5']

In [81]:
data['ret_dept_count'] = 0
for clust in cluster:
    inputlist= list(cluster_set[cluster_set['cluster'] == clust]['var_index'])
    cache = data.iloc[:,inputlist]
    pos = (cache>0).astype(int).sum(axis=1)
    neg = (cache<0).astype(int).sum(axis=1)
    data[clust+'_count'] = pos
    data['ret_dept_count'] = data['ret_dept_count'] + neg
    
    

In [138]:
data[0:2]

,TripType,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,...,cluster10_count,cluster8_count,cluster9_count,cluster2_count,cluster3_count,cluster1_count,cluster6_count,cluster7_count,cluster4_count,cluster5_count
0,999,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [161]:
data_test['ret_dept_count'] = 0
for clust in cluster:
    inputlist= list(cluster_set[cluster_set['cluster'] == clust]['var_index'])
    cache = data_test.iloc[:,inputlist]
    pos = (cache>0).astype(int).sum(axis=1)
    neg = (cache<0).astype(int).sum(axis=1)
    data_test[clust+'_count'] = pos
    data_test['ret_dept_count'] = data_test['ret_dept_count'] + neg

In [163]:
data_test = data_test.drop('TripType', axis=1)

In [164]:
data_test[0:2]

,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,...,cluster10_count,cluster8_count,cluster9_count,cluster2_count,cluster3_count,cluster1_count,cluster6_count,cluster7_count,cluster4_count,cluster5_count
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,2,1,0,0,0


In [142]:
data[0:5]

,TripType,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,...,cluster10_count,cluster8_count,cluster9_count,cluster2_count,cluster3_count,cluster1_count,cluster6_count,cluster7_count,cluster4_count,cluster5_count
0,999,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,26,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,3,1,1,0,0
3,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
4,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [84]:
data_x = data.drop('TripType', axis=1)
data_y = data['TripType']

In [85]:
# partition train and test set
n_row = data_x.shape[0]

np.random.seed(0)

row_ind = range(n_row)
np.random.shuffle(row_ind)

train_ind = row_ind[0:int(n_row*0.9)]
test_ind = row_ind[int(n_row*0.9):n_row]

train_x = data_x.iloc[train_ind, :]
train_y = data_y[train_ind]

test_x = data_x.iloc[test_ind, :]
test_y = data_y[test_ind]

In [169]:
# Gradient Boosting Classfier
clf = GradientBoostingClassifier(n_estimators = 150, max_leaf_nodes = None, max_depth = 4, learning_rate = 0.1, loss='deviance')

In [90]:
clf.fit(train_x, train_y)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=6, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=150,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [171]:
# log loss of test
clf_probs = clf.predict_proba(test_x)

score = log_loss(test_y, clf_probs)

y_pred = clf.predict(test_x)
acc = accuracy_score(test_y, y_pred)

print "Test accuracy =", acc, ", log loss =", score

Test accuracy = 0.746864548495 , log loss = 0.775786720207


In [170]:
clf.fit(data_x, data_y)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=4, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=150,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [172]:
# clf_probs = knn_f.predict_proba(data_test)
pred_probs = clf.predict_proba(data_test)
submission_probs = pd.DataFrame(pred_probs, columns = list(clf.classes_))

In [173]:
submission = pd.read_csv('sample_submission.csv', quotechar='"', skipinitialspace=True)

In [174]:
for i in range(len(submission_probs.columns)):
    # submission.iloc[:,[i+1]] = submission.iloc[:,[i+1]].astype(float)
    submission.iloc[:,[i+1]] = submission_probs.iloc[:,i]

In [175]:
submission.to_csv('submission_GBT_bucket.csv', sep=',', index = False)